In [ ]:
import auto_compyute as ac
import auto_compyute.nn.functional as F
from auto_compyute import nn

ac.backends.set_random_seed(0)
device = ac.cuda if ac.backends.gpu_available() else ac.cpu

In [ ]:
import pandas as pd

# download the datasets
# train_url = "https://pjreddie.com/media/files/mnist_train.csv"
train_data = pd.read_csv("../data/mnist_train.csv")
train_tensor = ac.tensor(train_data.to_numpy())

# test_url = "https://pjreddie.com/media/files/mnist_test.csv"
test_data = pd.read_csv("../data/mnist_test.csv")
test_tensor = ac.tensor(test_data.to_numpy())

In [ ]:
# split the data into train, val, test
rand_idx = ac.randperm(len(train_tensor))
n_train_samples = int(0.8 * len(train_tensor))
train_idx, val_idx = rand_idx[:n_train_samples], rand_idx[n_train_samples:]
train, val = train_tensor[train_idx], train_tensor[val_idx]

# split features from targets
X_train, y_train = train[:, 1:], train[:, 0].as_type(ac.int64)
X_val, y_val = val[:, 1:], val[:, 0].as_type(ac.int64)
X_test, y_test = test_tensor[:, 1:], test_tensor[:, 0].as_type(ac.int64)

# reshape the data into an image format (B, 784) -> (B, 1, 28, 28)
X_train = X_train.view((X_train.shape[0], 1 , 28, -1)).as_type(ac.float32)
X_val = X_val.view((X_val.shape[0], 1, 28, -1)).as_type(ac.float32)
X_test = X_test.view((X_test.shape[0], 1, 28, -1)).as_type(ac.float32)

# scaling
def scale(x: ac.Tensor) -> ac.Tensor:
    mean_px = x.mean().as_type(ac.float32)
    std_px = x.std().as_type(ac.float32)
    return (x - mean_px) / (std_px)

X_train = scale(X_train)
X_val = scale(X_val)
X_test = scale(X_test)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_val.shape=}')
print(f'{y_val.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

In [ ]:
model = nn.Sequential(
    nn.Conv2D(1, 32, 5), nn.ReLU(),
    nn.Conv2D(32, 32, 5, bias=False), nn.Batchnorm(32), nn.ReLU(),
    nn.MaxPooling2D(2), nn.Dropout(0.25),
    
    nn.Conv2D(32, 64, 3), nn.ReLU(),
    nn.Conv2D(64, 64, 3, bias=False), nn.Batchnorm(64), nn.ReLU(),
    nn.MaxPooling2D(2), nn.Dropout(0.25),

    nn.Flatten(),
    nn.Linear(576, 256, bias=False), nn.Batchnorm(256), nn.ReLU(),
    nn.Linear(256, 128, bias=False), nn.Batchnorm(128), nn.ReLU(),
    nn.Linear(128, 84, bias=False), nn.Batchnorm(84), nn.ReLU(), nn.Dropout(0.25),
    nn.Linear(84, 10),
)

model.to(device)

In [ ]:
batch_size = 256
train_dl = nn.Dataloader((X_train, y_train), batch_size=batch_size, device=device)
val_dl = nn.Dataloader((X_val, y_val), batch_size=batch_size, device=device)
train_steps = len(train_dl)

In [ ]:
# loss = F.cross_entropy(model(X_train[:256].to(ac.cuda)), y_train[:256].to(ac.cuda))
# ac.autograd.draw_compute_graph(loss)

In [ ]:
epochs = 1
lr = 1e-3
optim = nn.optimizers.Adam(model.parameters())

for e in range(1, epochs + 1):
    
    # training
    model.train()
    for s, (x, y) in enumerate(train_dl()):
        print(f"step {s}/{train_steps}", end="\r")
        F.cross_entropy(model(x), y).backward()
        optim.step()
        optim.zero_grad()
        break
    break
    
    # validiation
    model.eval()
    val_loss, val_acc = 0, 0
    with ac.no_grad():
        for x, y in val_dl():
            logits = model(x)
            val_loss += F.cross_entropy(logits, y).item()
            val_acc += (logits.data.argmax(-1) == y.data).mean()
    val_loss /= len(val_dl)
    val_acc /= len(val_dl)
    print(f"epoch {e}: {val_loss=:.4f} {val_acc=:.2f}")